In [43]:
import numpy as np
from dotenv import load_dotenv
from dwave.system import DWaveSampler, EmbeddingComposite
from numpy import hamming
from openjij import SQASampler
from pyqubo import Array, Constraint, LogEncInteger, solve_qubo
import math
import os
import dimod

In [44]:
load_dotenv()

TOKEN = os.getenv("TOKEN")
endpoint = "https://cloud.dwavesys.com/sapi/"

In [45]:
W = 750
cost = [135, 139, 149, 150, 156, 163, 173,
        184, 192, 201, 210, 214, 221, 229, 240]
weight = [70, 73, 77, 80, 82, 87, 90, 94, 98, 106, 110, 113, 115, 118, 120]
opt = [1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1]
c = {}
w = {}
N = len(cost)
for i in range(N):
    c[i] = cost[i]
    w[i] = weight[i]

In [46]:
x = Array.create('x', shape=(N), vartype='BINARY')
y = LogEncInteger("y", (0, W))

In [47]:
key1 = max(c, key=lambda k: c[k])
B = 1
A = B * c[key1] * 2

In [48]:
HA = Constraint(
    A * (W - sum(w[a] * x[a] for a in range(N)) - y)**2, label='HA'
)

HB = - B * sum(c[a] * x[a] for a in range(N))

In [49]:
H = HA + HB
model = H.compile()
Q, offset = model.to_qubo()

In [50]:
dw_sampler = DWaveSampler(solver='Advantage_system4.1',
                          token=TOKEN, endpoint=endpoint)
sampler = EmbeddingComposite(dw_sampler)
sampleset = sampler.sample_qubo(Q, num_reads=30)

In [51]:
sampleset.record

rec.array([([1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0], -2.69789346e+08, 1, 0.08),
           ([1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0], -2.69701026e+08, 1, 0.08),
           ([1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0], -2.68155811e+08, 1, 0.04),
           ([1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0], -2.64908522e+08, 1, 0.08),
           ([1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0], -2.65484533e+08, 1, 0.04),
           ([1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0], -2.65296362e+08, 1, 0.04),
           ([1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0], -2.60726797e+08, 1, 0.04),
           ([1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0], -2.58617190e+08, 1, 0.08),
           ([1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0,

In [52]:
decoded_sample = model.decode_sample(sampleset.first.sample, vartype="BINARY")

In [53]:
cost = 0
weight = 0
for k in range(N):
    if decoded_sample.array('x', k) != 0:
        cost += c[k]
        weight += w[k]
        

In [54]:
sol_y = sum(2**k * v for k, v in [(elem, decoded_sample.array('y', elem))
            for elem in range(math.ceil(math.log2(W)))])

In [55]:
print("スラック変数Y = {}".format(sol_y))
print()
print("broken")
print(decoded_sample.constraints(only_broken=True))
print("cost : "+str(cost))
print("weight : "+str(weight))

スラック変数Y = 202

broken
{'HA': (False, 1920.0)}
cost : 1063
weight : 550


In [56]:
init = sampleset.samples()[-1]

In [57]:
t = (0, 5, 15, 20)
s = (1.0, 0.4, 0.4, 1.0)

schedule = list(zip(t, s))

In [58]:
dw_sampler = DWaveSampler(solver='Advantage_system4.1',
                          token=TOKEN, endpoint=endpoint)
sampler = EmbeddingComposite(dw_sampler)
sampleset = sampler.sample_qubo(
    Q, num_reads=30, anneal_schedule=schedule, initial_state=init)

In [59]:
sampleset.record

rec.array([([0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0], -2.61889138e+08, 1, 0.12),
           ([0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0], -2.67372714e+08, 1, 0.08),
           ([0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0], -2.58319914e+08, 1, 0.12),
           ([0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0], -2.65296658e+08, 1, 0.08),
           ([0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0], -2.69845847e+08, 1, 0.12),
           ([0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0], -2.61763858e+08, 1, 0.04),
           ([0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0], -2.56129194e+08, 1, 0.16),
           ([0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0], -2.70000887e+08, 1, 0.12),
           ([0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,

In [60]:
decoded_sample = model.decode_sample(sampleset.first.sample, vartype="BINARY")

In [61]:
cost = 0
weight = 0
for k in range(N):
    if decoded_sample.array('x', k) != 0:
        cost += c[k]
        weight += w[k]

In [62]:
sol_y = sum(2**k * v for k, v in [(elem, decoded_sample.array('y', elem))
            for elem in range(math.ceil(math.log2(W)))])

In [63]:
print("スラック変数Y = {}".format(sol_y))
print()
print("broken")
print(decoded_sample.constraints(only_broken=True))
print("cost : "+str(cost))
print("weight : "+str(weight))

スラック変数Y = 41

broken
{'HA': (False, 480.0)}
cost : 1367
weight : 710
